# AUS Real Estate Sales September 2018 to June 2020
### Recorded property sales in major Australian Capital Cities
Kaggle: https://www.kaggle.com/htagholdings/aus-real-estate-sales-march-2019-to-april-2020

##### Columns:
1. date_sold - Date of sale
2. price - Price for which the property was sold for. Null if unknown.
3. suburb - Suburb of sale
4. city_name - Name of the city
5. state - State
6. lat - Latitude
7. lon - Longitude
8. bedrooms - Number of bedrooms
9. property_typ - Type of property i.e. house, unit, townhouse
10. oc_pid - ID of the locality (suburb)
11. lga_pid - ID of the LGA (council area)

In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
DATASET_PATH = 'datasets/aus-property-sales-sep2018-april2020.csv'
PREPARED_DATASET_PATH = 'datasets/aus-property-sales-sep2018-april2020_prepared.csv'
TRAIN_DATASET_PATH = 'datasets/aus-property-sales-sep2018-april2020_train.csv'
SCALER_FILE_PATH = 'scaler.pkl'
MODEL_FILE_PATH = 'models/model.pkl' 

### Loading dataset

In [3]:
df = pd.read_csv(DATASET_PATH)

In [4]:
df.shape

(320334, 11)

In [5]:
df.head()

,date_sold,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid
0,2018-09-18 00:00:00,NaN,Darling Point,Sydney,NSW,-33.869565,151.241317,3,unit,NSW1221,NSW180
1,2018-09-24 00:00:00,NaN,Darling Point,Sydney,NSW,-33.872179,151.239726,3,unit,NSW1221,NSW180
2,2018-09-26 00:00:00,1730000.0,Darling Point,Sydney,NSW,-33.868386,151.237471,2,unit,NSW1221,NSW180
3,2018-09-26 00:00:00,1928000.0,Darling Point,Sydney,NSW,-33.875465,151.236280,3,unit,NSW1221,NSW180
4,2018-10-02 00:00:00,1475000.0,Darling Point,Sydney,NSW,-33.875734,151.233575,3,unit,NSW1221,NSW180


In [6]:
df.tail()

,date_sold,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid
320329,2019-09-09 00:00:00,NaN,Taylor,Canberra,ACT,-35.152128,149.099435,3,townhouse,ACT934,ACT
320330,2019-09-10 00:00:00,549900.0,Taylor,Canberra,ACT,-35.151968,149.107229,3,townhouse,ACT934,ACT
320331,2019-09-10 00:00:00,499900.0,Taylor,Canberra,ACT,-35.151968,149.107229,3,townhouse,ACT934,ACT
320332,2020-03-10 00:00:00,434000.0,Taylor,Canberra,ACT,-35.152617,149.103578,3,townhouse,ACT934,ACT
320333,2020-05-19 00:00:00,420000.0,Taylor,Canberra,ACT,-35.151741,149.102781,2,townhouse,ACT934,ACT


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320334 entries, 0 to 320333
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date_sold      320334 non-null  object 
 1   price          257826 non-null  float64
 2   suburb         320334 non-null  object 
 3   city_name      320334 non-null  object 
 4   state          320334 non-null  object 
 5   lat            320256 non-null  float64
 6   lon            320256 non-null  float64
 7   bedrooms       320334 non-null  int64  
 8   property_type  320334 non-null  object 
 9   loc_pid        320334 non-null  object 
 10  lga_pid        320334 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 26.9+ MB


### Filling NaN values

In [19]:
for i in df.index[df['lat'].isnull()]:
    loc_pid = df.iloc[i, 9]
    df.iat[i, 5] = df.loc[(df['loc_pid'] == loc_pid) & (~df['lat'].isnull()), 'lat'].median()

In [20]:
for i in df.index[df['lon'].isnull()]:
    loc_pid = df.iloc[i, 9]
    df.iat[i, 6] = df.loc[(df['loc_pid'] == loc_pid) & (~df['lon'].isnull()), 'lon'].median()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320334 entries, 0 to 320333
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date_sold      320334 non-null  object 
 1   price          257826 non-null  float64
 2   suburb         320334 non-null  object 
 3   city_name      320334 non-null  object 
 4   state          320334 non-null  object 
 5   lat            320333 non-null  float64
 6   lon            320333 non-null  float64
 7   bedrooms       320334 non-null  int64  
 8   property_type  320334 non-null  object 
 9   loc_pid        320334 non-null  object 
 10  lga_pid        320334 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 26.9+ MB


In [24]:
df.loc[df['lat'].isnull()] = df['lat'].median()

In [25]:
df.loc[df['lon'].isnull()] = df['lon'].median()

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320334 entries, 0 to 320333
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date_sold      320334 non-null  object 
 1   price          257826 non-null  float64
 2   suburb         320334 non-null  object 
 3   city_name      320334 non-null  object 
 4   state          320334 non-null  object 
 5   lat            320334 non-null  float64
 6   lon            320334 non-null  float64
 7   bedrooms       320334 non-null  float64
 8   property_type  320334 non-null  object 
 9   loc_pid        320334 non-null  object 
 10  lga_pid        320334 non-null  object 
dtypes: float64(4), object(7)
memory usage: 26.9+ MB


In [27]:
for i in df.index[df['price'].isnull()]:
    loc_pid = df.iloc[i, 9]
    df.iat[i, 1] = df.loc[(df['loc_pid'] == loc_pid) & (~df['price'].isnull()), 'price'].median()

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320334 entries, 0 to 320333
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date_sold      320334 non-null  object 
 1   price          320313 non-null  float64
 2   suburb         320334 non-null  object 
 3   city_name      320334 non-null  object 
 4   state          320334 non-null  object 
 5   lat            320334 non-null  float64
 6   lon            320334 non-null  float64
 7   bedrooms       320334 non-null  float64
 8   property_type  320334 non-null  object 
 9   loc_pid        320334 non-null  object 
 10  lga_pid        320334 non-null  object 
dtypes: float64(4), object(7)
memory usage: 26.9+ MB


In [29]:
df.loc[df['price'].isnull()] = df['price'].median()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320334 entries, 0 to 320333
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date_sold      320334 non-null  object 
 1   price          320334 non-null  float64
 2   suburb         320334 non-null  object 
 3   city_name      320334 non-null  object 
 4   state          320334 non-null  object 
 5   lat            320334 non-null  float64
 6   lon            320334 non-null  float64
 7   bedrooms       320334 non-null  float64
 8   property_type  320334 non-null  object 
 9   loc_pid        320334 non-null  object 
 10  lga_pid        320334 non-null  object 
dtypes: float64(4), object(7)
memory usage: 26.9+ MB


In [31]:
df.head()

,date_sold,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid
0,2018-09-18 00:00:00,1675000.0,Darling Point,Sydney,NSW,-33.869565,151.241317,3.0,unit,NSW1221,NSW180
1,2018-09-24 00:00:00,1675000.0,Darling Point,Sydney,NSW,-33.872179,151.239726,3.0,unit,NSW1221,NSW180
2,2018-09-26 00:00:00,1730000.0,Darling Point,Sydney,NSW,-33.868386,151.237471,2.0,unit,NSW1221,NSW180
3,2018-09-26 00:00:00,1928000.0,Darling Point,Sydney,NSW,-33.875465,151.236280,3.0,unit,NSW1221,NSW180
4,2018-10-02 00:00:00,1475000.0,Darling Point,Sydney,NSW,-33.875734,151.233575,3.0,unit,NSW1221,NSW180


In [32]:
df.to_csv(PREPARED_DATASET_PATH)